In [1]:
!pip install "dask[complete]"
!pip install pyarrow pandas
!pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 131.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [4]:
data_dir = Path('/content/drive/MyDrive/Data_hasking')
ddf = dd.read_parquet(str(data_dir / '*.parquet'))
df = ddf.compute()

In [ ]:
pd.set_option('display.max_colwidth', None)

In [5]:
df

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,a168d638771f6240e978956399d9f16621b02655b20ae8...,CARD,432a9ae3afb5774dbb60c0f091dbd08193eaa4161d08ef...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,202308011836871,VND,-9600.0,-9600.0,110000,...,<NA>,Y,TP137,3006ab7cb6dc7542968e1e22dc0167a1d5b40a5463091a...,POS,HOI SO SHB,CM,<NA>,5590.91,20230801
1,4fd201b58b6b81ad3366160ff82f3d53f622e0290a57db...,CARD,3d1eaec79cbbaad7f4b8019b874ab4fe327f490e32ab4c...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,202308011836872,VND,-100.0,-100.0,110000,...,<NA>,Y,TP900,afcd5e115702ec6f7e4ea75699f88062b34506ff0c9ce6...,ATM,HOI SO SHB,CM,<NA>,6065.63,20230801
2,52dcdc6af49899ae1dfae36d4eee1625fe717ac6897c2b...,CARD,8c3b276a636351fae92fe15ec81a2c1835f54f394fea69...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,202308011836873,VND,-99.2,-99.2,110000,...,<NA>,Y,TP501,53f761a09578c10e92ec4378506860638bb0fd5b45e2c8...,EPOS,HOI SO SHB,CM,<NA>,5425.42,20230801
3,db34f14574441e3f42d7ea1d5299204ea481f9ac8f5ff5...,CARD,a7c0264cb2746c27921013f13b7326c2a1d4e2a7ec41ad...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,202308011836874,VND,-120.0,-120.0,110000,...,<NA>,Y,TP900,e5524afd37bd3778471ce72c4b1fc851928895d9cc1535...,ATM,HOI SO SHB,CM,<NA>,6061.19,20230801
4,49454b2d0968a2714066486a540ac8f0a57ee432cc3ca4...,CARD,41f062e47151420fa96091222e6be31458d670f623edae...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,202308011836875,VND,-120.0,-120.0,110000,...,<NA>,Y,TP900,3e3888614d3fce7b4899b7d28c8fed122efc90fcbd74c2...,ATM,HOI SO SHB,CM,<NA>,6013.04,20230801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031059,e3cc0b5bf27bfce6ed4bae39202055830d4a6b1cb91285...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2024013041306186,VND,-240.0,-240.0,110000,...,<NA>,N,1|2,a4c45b3a07c561f53d9c7ad994d56d389e6cb5c746ab16...,MOB,HOI SO SHB,FP,<NA>,NaN,20240130
1031060,e3cc0b5bf27bfce6ed4bae39202055830d4a6b1cb91285...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2024013041174162,VND,13387.1,13387.1,110000,...,<NA>,Y,1|2,81aa9a83bc3deb5f303dd3eec89f3b690de94dd211ebd7...,NET,HOI SO SHB,A2,SHB XXXXXXX,NaN,20240130
1031061,c4ab95caa9eca8c817385d55950edfb5fc17aad21295ec...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2024013041298813,VND,150.0,150.0,110000,...,XXXXXXXXXXXXXXXX PHAM THU T,Y,1|1,1742a8687fe4843a1646fd637ff601f390f59107a2902c...,NET,HOI SO SHB,FP,<NA>,NaN,20240130
1031062,9f0faaf612e947a6c27b95f3c0f10bb704b6e589d170fd...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2024013041175379,VND,2000.0,2000.0,110000,...,<NA>,N,1|2,78476611f44a393a4a6a1f575fbce648b54e7b76b005e7...,MOB,HOI SO SHB,A2,PHAM THI MAI LAN,NaN,20240130


In [6]:
df.shape

(139389125, 26)

In [ ]:
valeurs_manquantes = df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO                      0
ACC_BUSINESSTYPE           1325467
ACC_ACCNO                        0
ACC_BUSINESSNO                   0
ACC_CURRENCYISO            1325467
BUSINESSNO_TRANS                 0
FK_CURRENCY                      0
AMOUNT                           0
AMOUNTORIG                       0
BRANCH_OFFICE                    0
CONTRA_ACCNO                     0
CONTRA_ZIP               128047268
CONTRA_NAME              128043726
CSHYN                            0
REASON1                     717101
REASON2                   37025377
REASON4                  115787808
TR_FLAG_01                       0
ANALYTICAL_TRANS_CODE            0
TR_SP_01                         0
TR_SP_02                         0
TR_SP_03                         0
TR_SP_04                         0
TR_SP_05                 128043726
TR_SP_10                 139257171
VALUEDATE                        0
dtype: int64

In [ ]:
for column in df.columns:
    if pd.api.types.is_string_dtype(df[column]):
        df[column] = df[column].fillna('')
    elif pd.api.types.is_numeric_dtype(df[column]):
        df[column] = df[column].fillna(0)

In [ ]:
valeurs_manquantes = df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
REASON4                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
TR_SP_10                 0
VALUEDATE                0
dtype: int64

In [ ]:
label_encoders = {}
list_columns = ['ACC_BUSINESSTYPE', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'TR_SP_03', 'TR_SP_04', 'TR_SP_05', 'TR_SP_10']

In [ ]:
for column in list_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [ ]:

string_columns = [col for col in df.columns if pd.api.types.is_string_dtype(df[col])]
string_columns_without_list = [col for col in string_columns if col not in list_columns]
string_columns_without_list


['CUST_CUSTNO',
 'ACC_ACCNO',
 'ACC_BUSINESSNO',
 'BUSINESSNO_TRANS',
 'FK_CURRENCY',
 'BRANCH_OFFICE',
 'CONTRA_ACCNO',
 'CONTRA_ZIP',
 'CONTRA_NAME',
 'CSHYN',
 'REASON1',
 'REASON2',
 'REASON4',
 'TR_FLAG_01',
 'ANALYTICAL_TRANS_CODE',
 'VALUEDATE']

In [ ]:
label_encoders = {}

for column in string_columns_without_list:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [ ]:
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,635105,1,640823,1786,14,675461,34,-9600.0,-9600.0,0,...,0,1,78,22799471,13,66,7,0,21825,0
1,314095,1,583130,1786,14,675462,34,-100.0,-100.0,0,...,0,1,97,83444437,0,66,7,0,40448,0
2,326117,1,1334129,1786,14,675463,34,-99.2,-99.2,0,...,0,1,84,39861325,5,66,7,0,12233,0
3,863086,1,1596412,1786,14,675464,34,-120.0,-120.0,0,...,0,1,97,108854436,0,66,7,0,40007,0
4,288359,1,629035,1786,14,675465,34,-120.0,-120.0,0,...,0,1,97,29538690,0,66,7,0,35207,0


In [ ]:
# Apply FP-Growth algorithm
frequent_itemsets= fpgrowth(df.applymap(lambda x: 1 if x else 0), min_support=0.9, use_colnames=True)

In [ ]:
frequent_itemsets

,support,itemsets
0,1.000000,(TR_SP_01)
1,1.000000,(BUSINESSNO_TRANS)
2,1.000000,(FK_CURRENCY)
3,1.000000,(ACC_ACCNO)
4,1.000000,(ACC_BUSINESSNO)
...,...,...
80890,0.994545,"(ACC_BUSINESSNO, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, VALUEDATE, FK_CURRENCY, CUST_CUSTNO)"
80891,0.994545,"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, VALUEDATE, BUSINESSNO_TRANS, CUST_CUSTNO)"
80892,0.994545,"(ACC_BUSINESSNO, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, CUST_CUSTNO)"
80893,0.994544,"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, VALUEDATE, FK_CURRENCY, CUST_CUSTNO)"


In [ ]:
# Generate association rules with a confidence threshold of 1.0
rules0 = association_rules(frequent_itemsets, metric="confidence", min_threshold=1)

In [ ]:
rules0

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(REASON1, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.0,0.985363,1.0,1.0,1.413830e-08,inf,9.802682e-07
1,"(REASON1, TR_SP_01, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.0,0.985363,1.0,1.0,1.413830e-08,inf,9.802677e-07
2,"(BUSINESSNO_TRANS, REASON1, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.0,0.985363,1.0,1.0,1.413830e-08,inf,9.802677e-07
3,"(BUSINESSNO_TRANS, REASON1, TR_SP_01, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.0,0.985363,1.0,1.0,1.413830e-08,inf,9.802672e-07
4,"(REASON1, ACC_ACCNO, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.0,0.985363,1.0,1.0,1.413830e-08,inf,9.802658e-07
...,...,...,...,...,...,...,...,...,...,...
169275,"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, TR_SP_03)",(BUSINESSNO_TRANS),0.994545,1.0,0.994545,1.0,1.0,7.135022e-09,inf,1.315040e-06
169276,"(ACC_BUSINESSNO, ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, TR_SP_03)",(BUSINESSNO_TRANS),0.994545,1.0,0.994545,1.0,1.0,7.135022e-09,inf,1.315037e-06
169277,"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, TR_SP_03)",(BUSINESSNO_TRANS),0.994545,1.0,0.994545,1.0,1.0,7.135022e-09,inf,1.315035e-06
169278,"(ACC_BUSINESSNO, ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, FK_CURRENCY, TR_SP_03)",(BUSINESSNO_TRANS),0.994545,1.0,0.994545,1.0,1.0,7.135022e-09,inf,1.315033e-06


In [ ]:
# lower the value of confidence

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BUSINESSNO_TRANS),(TR_SP_01),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00
1,(TR_SP_01),(BUSINESSNO_TRANS),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00
2,(FK_CURRENCY),(BUSINESSNO_TRANS),1.000000,1.000000,1.000000,1.000000,1.000000,-1.110223e-16,1.000000,-7.737651e-09
3,(BUSINESSNO_TRANS),(FK_CURRENCY),1.000000,1.000000,1.000000,1.000000,1.000000,-1.110223e-16,1.000000,-1.547530e-08
4,(FK_CURRENCY),(TR_SP_01),1.000000,1.000000,1.000000,1.000000,1.000000,-1.110223e-16,1.000000,-7.737651e-09
...,...,...,...,...,...,...,...,...,...,...
48356799,(TR_SP_03),"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, CUST_CUSTNO)",0.999981,0.994563,0.994544,0.994563,1.000000,-1.684020e-08,0.999997,-8.935610e-04
48356800,(VALUEDATE),"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, FK_CURRENCY, BUSINESSNO_TRANS, CUST_CUSTNO)",0.995147,0.999393,0.994544,0.999394,1.000001,1.372426e-06,1.002277,2.843715e-04
48356801,(FK_CURRENCY),"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, VALUEDATE, BUSINESSNO_TRANS, CUST_CUSTNO)",1.000000,0.994545,0.994544,0.994545,1.000000,-7.827716e-11,1.000000,-5.455493e-03
48356802,(BUSINESSNO_TRANS),"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, VALUEDATE, FK_CURRENCY, CUST_CUSTNO)",1.000000,0.994544,0.994544,0.994545,1.000000,7.135022e-09,1.000001,1.000000e+00


In [ ]:
# filter rules with REASON1,CUST_CUSTNO,VALUEDATE

rule1 = rules[rules['antecedents'].apply(lambda x: 'REASON1' in x and 'CUST_CUSTNO' in x and 'VALUEDATE' in x)]
rule1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
134802,"(REASON1, CUST_CUSTNO, VALUEDATE)",(ACC_BUSINESSTYPE),0.990030,0.990491,0.980537,0.990412,0.999920,-7.817680e-05,0.991764,-7.933347e-03
140458,"(FK_CURRENCY, REASON1, CUST_CUSTNO, VALUEDATE)",(ACC_BUSINESSTYPE),0.990030,0.990491,0.980537,0.990412,0.999920,-7.817680e-05,0.991764,-7.933347e-03
140460,"(REASON1, CUST_CUSTNO, VALUEDATE, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.980537,1.000000,0.980537,1.000000,1.000000,1.406907e-08,inf,7.372272e-07
140464,"(REASON1, CUST_CUSTNO, VALUEDATE)","(FK_CURRENCY, ACC_BUSINESSTYPE)",0.990030,0.990491,0.980537,0.990412,0.999920,-7.816259e-05,0.991766,-7.931917e-03
140488,"(BUSINESSNO_TRANS, REASON1, CUST_CUSTNO, VALUEDATE)",(ACC_BUSINESSTYPE),0.990030,0.990491,0.980537,0.990412,0.999920,-7.817680e-05,0.991764,-7.933347e-03
...,...,...,...,...,...,...,...,...,...,...
48318202,"(REASON1, ACC_ACCNO, CUST_CUSTNO, VALUEDATE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_03)",0.990030,0.999394,0.989428,0.999393,0.999998,-1.656624e-06,0.997246,-1.679063e-04
48318208,"(REASON1, TR_SP_03, VALUEDATE, CUST_CUSTNO)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, FK_CURRENCY, BUSINESSNO_TRANS)",0.990011,0.999413,0.989428,0.999412,0.999998,-1.565352e-06,0.997313,-1.583582e-04
48318213,"(FK_CURRENCY, REASON1, CUST_CUSTNO, VALUEDATE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, BUSINESSNO_TRANS, TR_SP_03)",0.990030,0.999394,0.989428,0.999392,0.999998,-1.671166e-06,0.997222,-1.693805e-04
48318214,"(BUSINESSNO_TRANS, REASON1, CUST_CUSTNO, VALUEDATE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, FK_CURRENCY, TR_SP_03)",0.990030,0.999394,0.989428,0.999392,0.999998,-1.664063e-06,0.997233,-1.686607e-04


In [ ]:
# Define thresholds for filtering
confidence_threshold = 0.9
lift_threshold = 1
leverage_threshold = 0  # Close to 0

# Ensure leverage is a positive value close to 0
leverage_lower_bound = -0.01
leverage_upper_bound = 0.01

# Filter rules based on defined criteria, using the 'rules' DataFrame directly
filtered_rules = rules[
    (rules['confidence'] >= confidence_threshold) &
    (rules['lift'] >= lift_threshold) &
    (rules['leverage'] >= leverage_lower_bound) &
    (rules['leverage'] <= leverage_upper_bound)
]

filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BUSINESSNO_TRANS),(TR_SP_01),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00
1,(TR_SP_01),(BUSINESSNO_TRANS),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00
57008,"(REASON1, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.000000,0.985363,1.000000,1.000000,1.413830e-08,inf,9.802682e-07
57009,(FK_CURRENCY),"(REASON1, ACC_BUSINESSTYPE)",1.000000,0.985363,0.985363,0.985363,1.000000,1.413830e-08,1.000001,1.000000e+00
57069,"(REASON1, TR_SP_01, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.985363,1.000000,0.985363,1.000000,1.000000,1.413830e-08,inf,9.802677e-07
...,...,...,...,...,...,...,...,...,...,...
48356790,"(CUST_CUSTNO, VALUEDATE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_03)",0.995146,0.999394,0.994544,0.999396,1.000001,1.378062e-06,1.002291,2.854613e-04
48356791,"(FK_CURRENCY, BUSINESSNO_TRANS)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, TR_SP_03)",1.000000,0.994545,0.994544,0.994545,1.000000,7.056745e-09,1.000001,3.296764e-01
48356797,(ANALYTICAL_TRANS_CODE),"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ACC_ACCNO, TR_SP_03, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, CUST_CUSTNO)",0.999414,0.995127,0.994544,0.995128,1.000001,1.367700e-06,1.000281,2.345011e-03
48356800,(VALUEDATE),"(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, ACC_ACCNO, TR_SP_03, FK_CURRENCY, BUSINESSNO_TRANS, CUST_CUSTNO)",0.995147,0.999393,0.994544,0.999394,1.000001,1.372426e-06,1.002277,2.843715e-04


In [ ]:


filtered_rules[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].head(10)


,antecedents,consequents,confidence,lift,leverage,conviction
0,(BUSINESSNO_TRANS),(TR_SP_01),1.000000,1.0,0.000000e+00,1.000000
1,(TR_SP_01),(BUSINESSNO_TRANS),1.000000,1.0,0.000000e+00,1.000000
57008,"(REASON1, ACC_BUSINESSTYPE)",(FK_CURRENCY),1.000000,1.0,1.413830e-08,inf
57009,(FK_CURRENCY),"(REASON1, ACC_BUSINESSTYPE)",0.985363,1.0,1.413830e-08,1.000001
57069,"(REASON1, TR_SP_01, ACC_BUSINESSTYPE)",(FK_CURRENCY),1.000000,1.0,1.413830e-08,inf
57071,"(FK_CURRENCY, TR_SP_01)","(REASON1, ACC_BUSINESSTYPE)",0.985363,1.0,1.403329e-08,1.000001
57074,"(REASON1, ACC_BUSINESSTYPE)","(FK_CURRENCY, TR_SP_01)",1.000000,1.0,1.403329e-08,2.956089
57076,(FK_CURRENCY),"(REASON1, TR_SP_01, ACC_BUSINESSTYPE)",0.985363,1.0,1.413830e-08,1.000001
57082,"(BUSINESSNO_TRANS, REASON1, ACC_BUSINESSTYPE)",(FK_CURRENCY),1.000000,1.0,1.413830e-08,inf
57086,"(REASON1, ACC_BUSINESSTYPE)","(FK_CURRENCY, BUSINESSNO_TRANS)",1.000000,1.0,1.403329e-08,2.956089


In [ ]:
# prompt: select the rule having conviction1 and lift 1 and containing VALUEDATE

filtered_rules[ (filtered_rules['conviction'] >=1) & (filtered_rules['lift']>= 1) & (filtered_rules['antecedents'].apply(lambda x: 'VALUEDATE' in x)) ].head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
134720,"(REASON1, VALUEDATE, ACC_BUSINESSTYPE)",(BUSINESSNO_TRANS),0.980539,1.0,0.980539,1.0,1.0,7.034543e-09,inf,3.686387e-07
134735,"(REASON1, VALUEDATE, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.980539,1.0,0.980539,1.0,1.0,1.406909e-08,inf,7.372775e-07
134845,"(FK_CURRENCY, REASON1, VALUEDATE, ACC_BUSINESSTYPE)",(BUSINESSNO_TRANS),0.980539,1.0,0.980539,1.0,1.0,7.034543e-09,inf,3.686387e-07
134846,"(BUSINESSNO_TRANS, REASON1, VALUEDATE, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.980539,1.0,0.980539,1.0,1.0,1.406909e-08,inf,7.372775e-07
134851,"(REASON1, VALUEDATE, ACC_BUSINESSTYPE)","(FK_CURRENCY, BUSINESSNO_TRANS)",0.980539,1.0,0.980539,1.0,1.0,2.110363e-08,inf,1.105916e-06
134875,"(REASON1, TR_SP_01, VALUEDATE, ACC_BUSINESSTYPE)",(FK_CURRENCY),0.980539,1.0,0.980539,1.0,1.0,1.406909e-08,inf,7.372772e-07
134886,"(REASON1, VALUEDATE, ACC_BUSINESSTYPE)","(FK_CURRENCY, TR_SP_01)",0.980539,1.0,0.980539,1.0,1.0,1.392947e-08,2.941616,7.299609e-07
134905,"(REASON1, TR_SP_01, VALUEDATE, ACC_BUSINESSTYPE)",(BUSINESSNO_TRANS),0.980539,1.0,0.980539,1.0,1.0,7.034543e-09,inf,3.686386e-07
134916,"(REASON1, VALUEDATE, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, TR_SP_01)",0.980539,1.0,0.980539,1.0,1.0,6.894925e-09,1.961078,3.613222e-07
134935,"(TR_SP_01, REASON1, VALUEDATE, FK_CURRENCY, ACC_BUSINESSTYPE)",(BUSINESSNO_TRANS),0.980539,1.0,0.980539,1.0,1.0,7.034543e-09,inf,3.686386e-07


In [ ]:
# prompt: filter rules with CUST_CUSTNO, ACC_BUSINESSTYPE,VALUEDATE,AMOUNT

rule2 = filtered_rules[filtered_rules['antecedents'].apply(lambda x: 'CUST_CUSTNO' in x and 'ACC_BUSINESSTYPE' in x and 'VALUEDATE' in x and 'AMOUNT' in x)]
rule2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6060440,"(AMOUNT, CUST_CUSTNO, VALUEDATE, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),0.963153,0.990491,0.963153,1.000000,1.009600,9.158728e-03,inf,2.580682e-01
6060470,"(AMOUNT, CUST_CUSTNO, VALUEDATE, ACC_BUSINESSTYPE)",(BUSINESSNO_TRANS),0.963153,1.000000,0.963153,1.000000,1.000000,6.909812e-09,inf,1.946998e-07
6060650,"(AMOUNT, VALUEDATE, ACC_CURRENCYISO, CUST_CUSTNO, ACC_BUSINESSTYPE)",(BUSINESSNO_TRANS),0.963153,1.000000,0.963153,1.000000,1.000000,6.909812e-09,inf,1.946998e-07
6060652,"(AMOUNT, VALUEDATE, BUSINESSNO_TRANS, CUST_CUSTNO, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),0.963153,0.990491,0.963153,1.000000,1.009600,9.158728e-03,inf,2.580682e-01
6060662,"(AMOUNT, CUST_CUSTNO, VALUEDATE, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, ACC_CURRENCYISO)",0.963153,0.990491,0.963153,1.000000,1.009600,9.158735e-03,inf,2.580684e-01
...,...,...,...,...,...,...,...,...,...,...
39927023,"(AMOUNT, AMOUNTORIG, ACC_CURRENCYISO, VALUEDATE, CUST_CUSTNO, ACC_BUSINESSTYPE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, REASON1, ACC_ACCNO, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03)",0.963153,0.974745,0.938917,0.974837,1.000095,8.910868e-05,1.003677,2.575643e-03
39927043,"(AMOUNT, CUST_CUSTNO, AMOUNTORIG, VALUEDATE, TR_SP_03, ACC_BUSINESSTYPE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, REASON1, ACC_ACCNO, ACC_CURRENCYISO, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02)",0.963134,0.965284,0.938917,0.974857,1.009917,9.219525e-03,1.380712,2.663497e-01
39927059,"(AMOUNT, AMOUNTORIG, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, REASON1, ACC_ACCNO, ACC_CURRENCYISO, TR_SP_02, BUSINESSNO_TRANS, TR_SP_03)",0.963153,0.965265,0.938917,0.974837,1.009917,9.219582e-03,1.380419,2.664874e-01
39927062,"(AMOUNT, AMOUNTORIG, VALUEDATE, BUSINESSNO_TRANS, CUST_CUSTNO, ACC_BUSINESSTYPE)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ANALYTICAL_TRANS_CODE, REASON1, ACC_ACCNO, ACC_CURRENCYISO, FK_CURRENCY, TR_SP_02, TR_SP_03)",0.963153,0.965265,0.938917,0.974837,1.009917,9.219568e-03,1.380418,2.664871e-01


In [ ]:
# filter the top 10 rules by lift and conviction

rule2[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].sort_values(by=['lift', 'conviction'], ascending=False).head(10)


,antecedents,consequents,confidence,lift,leverage,conviction
6549192,"(AMOUNT, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6550595,"(AMOUNT, TR_SP_01, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6553148,"(AMOUNT, ACC_ACCNO, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6555198,"(AMOUNT, TR_SP_01, ACC_ACCNO, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6550714,"(AMOUNT, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, TR_SP_01, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323943
6557587,"(AMOUNT, ACC_BUSINESSNO, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6560399,"(AMOUNT, ACC_BUSINESSNO, TR_SP_01, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6555388,"(AMOUNT, ACC_ACCNO, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, TR_SP_01, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323943
6565512,"(AMOUNT, ACC_BUSINESSNO, ACC_ACCNO, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947
6569611,"(AMOUNT, ACC_BUSINESSNO, TR_SP_01, ACC_ACCNO, VALUEDATE, FK_CURRENCY, CUST_CUSTNO, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, REASON1, ACC_CURRENCYISO)",0.995596,1.010386,0.009857,3.323947


In [ ]:
# prompt: filter rules with FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03 top 10 by lift and conviction

rule2[rule2['antecedents'].apply(lambda x: 'FK_CURRENCY' in x and 'BUSINESSNO_TRANS' in x and 'TR_SP_02' in x and 'TR_SP_03' in x)][['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].sort_values(by=['lift', 'conviction'], ascending=False).head(10)


,antecedents,consequents,confidence,lift,leverage,conviction
10537337,"(AMOUNT, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10538867,"(AMOUNT, TR_SP_01, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10544987,"(AMOUNT, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10548053,"(AMOUNT, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10555451,"(AMOUNT, ACC_BUSINESSNO, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10558517,"(AMOUNT, ACC_BUSINESSNO, TR_SP_01, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10570781,"(AMOUNT, ACC_BUSINESSNO, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10576919,"(AMOUNT, ACC_BUSINESSNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10586619,"(AMOUNT, CONTRA_ACCNO, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf
10589685,"(AMOUNT, TR_SP_01, CONTRA_ACCNO, CUST_CUSTNO, VALUEDATE, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, TR_SP_03, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),1.0,1.0096,0.008969,inf


In [ ]:

rule3 = filtered_rules[filtered_rules['antecedents'].apply(lambda x: 'CUST_CUSTNO' in x and 'REASON1' in x and 'AMOUNT' in x)]
rule3


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4717621,"(AMOUNT, REASON1, CUST_CUSTNO, ACC_CURRENCYISO)",(FK_CURRENCY),0.963719,1.0,0.963719,1.0,1.0,1.382774e-08,inf,3.954731e-07
4717801,"(AMOUNT, TR_SP_01, REASON1, ACC_CURRENCYISO, CUST_CUSTNO)",(FK_CURRENCY),0.963719,1.0,0.963719,1.0,1.0,1.382774e-08,inf,3.954730e-07
4717813,"(AMOUNT, REASON1, CUST_CUSTNO, ACC_CURRENCYISO)","(FK_CURRENCY, TR_SP_01)",0.963719,1.0,0.963719,1.0,1.0,1.356746e-08,2.891156,3.880289e-07
4717864,"(AMOUNT, REASON1, ACC_CURRENCYISO, BUSINESSNO_TRANS, CUST_CUSTNO)",(FK_CURRENCY),0.963719,1.0,0.963719,1.0,1.0,1.382774e-08,inf,3.954730e-07
4717870,"(AMOUNT, REASON1, CUST_CUSTNO, ACC_CURRENCYISO)","(FK_CURRENCY, BUSINESSNO_TRANS)",0.963719,1.0,0.963719,1.0,1.0,1.356746e-08,2.891156,3.880289e-07
...,...,...,...,...,...,...,...,...,...,...
47413811,"(AMOUNT, ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, CUST_CUSTNO, AMOUNTORIG, REASON1, TR_SP_04, TR_SP_03)","(ACC_BUSINESSNO, TR_SP_01, ACC_ACCNO, FK_CURRENCY, BUSINESSNO_TRANS)",0.902396,1.0,0.902396,1.0,1.0,9.618674e-09,1.095767,1.092074e-07
47413885,"(AMOUNT, ANALYTICAL_TRANS_CODE, CUST_CUSTNO, AMOUNTORIG, REASON1, ACC_ACCNO, TR_SP_04, TR_SP_03)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, FK_CURRENCY, BUSINESSNO_TRANS)",0.902396,1.0,0.902396,1.0,1.0,2.937717e-07,3.729905,3.335391e-06
47413892,"(AMOUNT, ANALYTICAL_TRANS_CODE, CUST_CUSTNO, AMOUNTORIG, REASON1, FK_CURRENCY, TR_SP_04, TR_SP_03)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ACC_ACCNO, BUSINESSNO_TRANS)",0.902396,1.0,0.902396,1.0,1.0,2.773227e-07,2.932788,3.148636e-06
47413893,"(AMOUNT, ANALYTICAL_TRANS_CODE, CUST_CUSTNO, AMOUNTORIG, REASON1, BUSINESSNO_TRANS, TR_SP_04, TR_SP_03)","(ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ACC_ACCNO, FK_CURRENCY)",0.902396,1.0,0.902396,1.0,1.0,2.837966e-07,2.977908,3.222139e-06


In [ ]:
# prompt: filter the rule3 having a lot of items in antecedants and consequents display the top 10

rule3[rule3['antecedents'].apply(lambda x: len(x) > 2) & rule3['consequents'].apply(lambda x: len(x) > 2)][['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].sort_values(by=['lift', 'conviction'], ascending=False).head(10)


,antecedents,consequents,confidence,lift,leverage,conviction
11374526,"(AMOUNT, REASON1, VALUEDATE, ACC_CURRENCYISO, CUST_CUSTNO)","(FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, ACC_BUSINESSTYPE)",0.979774,1.009635,0.008965,1.462268
11374547,"(AMOUNT, REASON1, VALUEDATE, CUST_CUSTNO, ACC_BUSINESSTYPE)","(FK_CURRENCY, BUSINESSNO_TRANS, ACC_CURRENCYISO, TR_SP_02)",0.979774,1.009635,0.008965,1.462268
11377374,"(AMOUNT, TR_SP_01, REASON1, ACC_CURRENCYISO, VALUEDATE, CUST_CUSTNO)","(FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, ACC_BUSINESSTYPE)",0.979774,1.009635,0.008965,1.462268
11377395,"(AMOUNT, TR_SP_01, REASON1, VALUEDATE, CUST_CUSTNO, ACC_BUSINESSTYPE)","(FK_CURRENCY, BUSINESSNO_TRANS, ACC_CURRENCYISO, TR_SP_02)",0.979774,1.009635,0.008965,1.462268
11375832,"(AMOUNT, REASON1, VALUEDATE, ACC_CURRENCYISO, CUST_CUSTNO)","(FK_CURRENCY, TR_SP_01, TR_SP_02, ACC_BUSINESSTYPE)",0.979774,1.009635,0.008965,1.462267
11375845,"(AMOUNT, REASON1, VALUEDATE, CUST_CUSTNO, ACC_BUSINESSTYPE)","(FK_CURRENCY, TR_SP_02, TR_SP_01, ACC_CURRENCYISO)",0.979774,1.009635,0.008965,1.462267
11377446,"(AMOUNT, REASON1, VALUEDATE, ACC_CURRENCYISO, BUSINESSNO_TRANS, CUST_CUSTNO)","(FK_CURRENCY, TR_SP_01, TR_SP_02, ACC_BUSINESSTYPE)",0.979774,1.009635,0.008965,1.462267
11377469,"(AMOUNT, REASON1, VALUEDATE, BUSINESSNO_TRANS, CUST_CUSTNO, ACC_BUSINESSTYPE)","(FK_CURRENCY, TR_SP_02, TR_SP_01, ACC_CURRENCYISO)",0.979774,1.009635,0.008965,1.462267
11377640,"(AMOUNT, REASON1, VALUEDATE, ACC_CURRENCYISO, CUST_CUSTNO)","(TR_SP_01, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02, ACC_BUSINESSTYPE)",0.979774,1.009635,0.008965,1.462267
11377661,"(AMOUNT, REASON1, VALUEDATE, CUST_CUSTNO, ACC_BUSINESSTYPE)","(TR_SP_01, ACC_CURRENCYISO, FK_CURRENCY, BUSINESSNO_TRANS, TR_SP_02)",0.979774,1.009635,0.008965,1.462267


In [ ]:
# reverse encoding label

for column, le in label_encoders.items():
    df[column] = le.inverse_transform(df[column])
df.head()


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,a168d638771f6240e978956399d9f16621b02655b20ae81efcbeaf18af4cdc73,1,432a9ae3afb5774dbb60c0f091dbd08193eaa4161d08ef4fbcd6698608be9ff7,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011836871,VND,-9600.0,-9600.0,110000,...,,Y,TP137,22799471,13,66,7,0,21825,20230801
1,4fd201b58b6b81ad3366160ff82f3d53f622e0290a57dbe0b94e10c5274c88df,1,3d1eaec79cbbaad7f4b8019b874ab4fe327f490e32ab4cd0ac1deab84f26df36,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011836872,VND,-100.0,-100.0,110000,...,,Y,TP900,83444437,0,66,7,0,40448,20230801
2,52dcdc6af49899ae1dfae36d4eee1625fe717ac6897c2b411a30512c3ffdd834,1,8c3b276a636351fae92fe15ec81a2c1835f54f394fea69352f98d033b9e480a1,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011836873,VND,-99.2,-99.2,110000,...,,Y,TP501,39861325,5,66,7,0,12233,20230801
3,db34f14574441e3f42d7ea1d5299204ea481f9ac8f5ff5fb8b90e981ba14fb4a,1,a7c0264cb2746c27921013f13b7326c2a1d4e2a7ec41ad1ed58ec3e303612fa9,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011836874,VND,-120.0,-120.0,110000,...,,Y,TP900,108854436,0,66,7,0,40007,20230801
4,49454b2d0968a2714066486a540ac8f0a57ee432cc3ca429fb5afc086bfeacd0,1,41f062e47151420fa96091222e6be31458d670f623edaef9d7a19a166d2de281,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011836875,VND,-120.0,-120.0,110000,...,,Y,TP900,29538690,0,66,7,0,35207,20230801


In [ ]:
# prompt: valide some rules with examples from the data

# Select a rule from the filtered rules DataFrame
selected_rule = filtered_rules.iloc[0]

# Extract the antecedents and consequents of the selected rule
antecedents = selected_rule["antecedents"]
consequents = selected_rule["consequents"]

# Find examples from the data that satisfy the antecedents
antecedent_examples = df[df[list(antecedents)].all(axis=1)]

# Check if the consequents are also present in the examples
consequent_examples = antecedent_examples[antecedent_examples[list(consequents)].all(axis=1)]

# Print the examples
print("Examples where the antecedents and consequents are both present:")
consequent_examples


Examples where the antecedents and consequents are both present:


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
707,8d9c5a8a77a0bf19b9a2d80b2c6113daf14bb409c81961a5574da1f83bc76ffe,2,bdbf5c77c6c61ed23b00c5a20e692c35bf9f84be8142fbdca5a6fe3a619a37cc,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011381518,VND,1960.00,1960.00,110000,...,XXXXXXXXXXXXXXX NGUYEN VAN,Y,1|1,72065936,10,66,11,0,0,20230801
708,1860d0281f23ef94f964d0319158ca0ab52d8c049fc2116e6e1829373dac0431,2,66109d188b6363609804443fab8b631cfb5dc19a8d8bd8e83db478a740fc20ea,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011716362,VND,500.00,500.00,110000,...,RINH QUANG HUNG toi XXXXXXX,Y,1|1,111100317,10,66,11,0,0,20230801
709,af3e6c0fbc6207736f6c97671d63e75cfc109129d673881416e12fa47f8cf5d8,2,3fdb8894344ab1289cec0efdd03ef870cfc23459f2e961fc1b3158d54f078b94,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011219361,VND,414.50,414.50,110000,...,XX.XX.XXXX sXXXXXXXX bcXXXX,N,21|45,91176598,16,66,15,15989,0,20230801
710,a62c6725ace84f88ede723adfa0cfa135a636ead8e68ee6def19310a4ca22c0e,2,e142422e98c42fb33be64bbfcb0b7e6508905b4ba7742657e3258be5d946a1b7,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011390326,VND,500.00,500.00,110000,...,ap(GD#XXXXXX),Y,1|1,64584934,10,66,11,0,0,20230801
711,efb1b0665c8ee12f66830e3e28ff83a25068d3c1aee4e38f665f8c19e5193840,2,ec8b4e1fd937d3415f1050cdb14ac49c3ba75a282cfec62794f1623de2217583,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,202308011304721,VND,22800.00,22800.00,110000,...,XX),Y,1|1,74622055,10,66,11,0,0,20230801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009790,1b284fa11559c94406a6e250f036c09be613bbc99c73dcc8f3bbb7c42746fa27,6,3e8990702eef575c5579690b42fea11f592757391aad99ceca209ba7c1fc5309,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,2024013042076485,VND,-86393.03,-86393.03,110300,...,,N,12|15,106871756,2,48,20,0,0,20240130
1009791,1b284fa11559c94406a6e250f036c09be613bbc99c73dcc8f3bbb7c42746fa27,6,3e8990702eef575c5579690b42fea11f592757391aad99ceca209ba7c1fc5309,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,2024013042071928,VND,-0.06,-0.06,110300,...,,N,1|1,1089390,2,48,20,0,0,20240130
1009792,1b284fa11559c94406a6e250f036c09be613bbc99c73dcc8f3bbb7c42746fa27,6,3e8990702eef575c5579690b42fea11f592757391aad99ceca209ba7c1fc5309,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,2024013042081058,VND,86393.03,86393.03,110300,...,,N,1|1,106871756,2,48,20,0,0,20240130
1009794,1b284fa11559c94406a6e250f036c09be613bbc99c73dcc8f3bbb7c42746fa27,6,3e8990702eef575c5579690b42fea11f592757391aad99ceca209ba7c1fc5309,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,14,2024013042092574,VND,-86393.03,-86393.03,110300,...,,N,12|12,71733893,2,48,20,0,0,20240130


In [ ]:
selected_rule = filtered_rules.iloc[0]
selected_rule

antecedents               (ACC_BUSINESSNO, TR_SP_01, CONTRA_ACCNO, ACC_ACCNO, FK_CURRENCY)
consequents           (CUST_CUSTNO, REASON1, BUSINESSNO_TRANS, TR_SP_03, ACC_BUSINESSTYPE)
antecedent support                                                                     1.0
consequent support                                                                0.985343
support                                                                           0.985342
confidence                                                                        0.985343
lift                                                                                   1.0
leverage                                                                               0.0
conviction                                                                        1.000001
zhangs_metric                                                                     0.015878
Name: 82118, dtype: object

In [ ]:
# valide all filtered_rules

# Loop through each rule in the filtered_rules DataFrame
for index, rule in filtered_rules.iterrows():
    # Extract the antecedents and consequents of the rule
    antecedents = rule["antecedents"]
    consequents = rule["consequents"]

    # Find examples from the data that satisfy the antecedents
    antecedent_examples = df[df[list(antecedents)].all(axis=1)]

    # Check if the consequents are also present in the examples
    consequent_examples = antecedent_examples[antecedent_examples[list(consequents)].all(axis=1)]

    # Print the examples
    print(f"Rule: {rule}")
    print(f"Examples where the antecedents and consequents are both present:")
    consequent_examples
    print("\n")